![Team USA Football](images/Usa_football_body_logo.png)

Data Dictionary here https://nflreadr.nflverse.com/articles/dictionary_player_stats.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
stats_df = pd.read_csv('data/player_stats.csv')
qbr_df = pd.read_csv('data/qbr_season_level.csv') 

stats_df contains player stats by game ill need to get some season totals, career totals, as well as per game stats for passing receiving and rushing. ill use interceptions, special teams tds, and fumbles, sacks to help rank defense. the stats ill use arent missing many entries so after cleaning ill save what i need into a new dataframe. also need to have a cut off for age. if a player is XX y/o they wont be relevant today. ill try to expand on the EPA missing data as that is Expected Points Added which should be extremely beneficial to analysis

In [7]:
stats_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,yards_per_carry,yards_per_reception,tds_per_attempt,tds_per_carry,tds_per_reception,season_passing_yards,cumulative_rushing_yards,cumulative_receiving_yards,completions_per_attempt,yards_per_touch
0,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,1,REG,...,3.529412,3.500000,0.0,0.058824,0.0,0,60,7,0.0,3.722222
1,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,2,REG,...,3.300000,4.500000,0.0,0.000000,0.0,0,93,25,0.0,3.923077
2,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,4,REG,...,0.500000,0.000000,0.0,0.000000,0.0,0,95,25,0.0,0.500000
3,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,7,REG,...,3.857143,2.666667,0.0,0.000000,0.0,0,122,33,0.0,3.888889
4,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,8,REG,...,2.785714,0.000000,0.0,0.000000,0.0,0,161,33,0.0,2.785714


In [8]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129739 entries, 0 to 129738
Data columns (total 65 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   player_id                    129739 non-null  object 
 1   player_name                  61504 non-null   object 
 2   player_display_name          129736 non-null  object 
 3   position                     129667 non-null  object 
 4   position_group               129667 non-null  object 
 5   headshot_url                 69803 non-null   object 
 6   recent_team                  129739 non-null  object 
 7   season                       129739 non-null  int64  
 8   week                         129739 non-null  int64  
 9   season_type                  129739 non-null  object 
 10  opponent_team                129739 non-null  object 
 11  completions                  129739 non-null  int64  
 12  attempts                     129739 non-null  int64  
 13 

In [10]:
stats_df['season'].value_counts()

2021    5698
2023    5653
2022    5631
2020    5447
2012    5354
2014    5350
2017    5319
2015    5318
2011    5301
2018    5281
2016    5274
2019    5261
2009    5242
2013    5231
2010    5201
2003    5142
2004    5129
2002    5078
1999    5031
2001    4895
2000    4874
2007    4841
2008    4814
2006    4712
2005    4662
Name: season, dtype: int64

### Feature Engineering

In [6]:
# Create efficiency metrics
stats_df['completion_percentage'] = stats_df['completions'] / (stats_df['attempts'] + 1)
stats_df['yards_per_attempt'] = stats_df['passing_yards'] / (stats_df['attempts'] + 1)
stats_df['yards_per_carry'] = stats_df['rushing_yards'] / (stats_df['carries'] + 1)
stats_df['yards_per_reception'] = stats_df['receiving_yards'] / (stats_df['receptions'] + 1)

# Create touchdown ratios
stats_df['tds_per_attempt'] = stats_df['passing_tds'] / (stats_df['attempts'] + 1)
stats_df['tds_per_carry'] = stats_df['rushing_tds'] / (stats_df['carries'] + 1)
stats_df['tds_per_reception'] = stats_df['receiving_tds'] / (stats_df['receptions'] + 1)

# Calculate cumulative stats for each player over the season
stats_df['season_passing_yards'] = stats_df.groupby(['player_id', 'season'])['passing_yards'].transform('sum')
stats_df['cumulative_rushing_yards'] = stats_df.groupby(['player_id', 'season'])['rushing_yards'].cumsum()
stats_df['cumulative_receiving_yards'] = stats_df.groupby(['player_id', 'season'])['receiving_yards'].cumsum()

# Interaction terms
stats_df['completions_per_attempt'] = stats_df['completions'] / (stats_df['attempts'] + 1)
stats_df['yards_per_touch'] = (stats_df['rushing_yards'] + stats_df['receiving_yards']) / (stats_df['carries'] + stats_df['receptions'] + 1)

## QBR

not sure if this will even be necessary. might be good to have QBR by season to go along with other QB stats

In [ ]:
qbr_df.info()

In [ ]:
qbr_df['season'].value_counts()